In [ ]:
import asyncio
import aiohttp

## Test the function below to see if it can handle the api requests correctly 

async def test_single_api_call(session, url, parameter, name):
    try:
        param = {parameter: name}
        response = await session.get(url=url, params=param)
        if response.status == 200:
            # Request ok
            return(response.status, await response.json())

        else:
            return(response.status, None)
    except aiohttp.ClientError as e:
        print(f"A client error occured: {e}.")

async def testing_suite():
    async with aiohttp.ClientSession() as session:
        ## The site for making api requests contains jsons for TV shows. ##
        url = "https://api.tvmaze.com/singlesearch/shows"
        # Replace show name with the whole or partial name of a show that exists,  
        ## since the website supports fuzzy search
        ## expect status code 200
        show_name = "game thrones"  # Shows we know exist
        request_status, request_info, = await test_single_api_call(session, url, "q", show_name)
        assert request_status == 200 
        assert 'name' in request_info # the json should contain the key 'name'
        
        ## No show contain all the words below, expect status code 404
        show_name = "A show that really does not exist"  
        request_status, request_info, = await test_single_api_call(session, url, "q", show_name)
        assert request_status == 404
        assert request_info is None
        
        ## check if the function handles invalid url correctly
        try:
            invalid_url = "not_an_url"
            show_name = "game of thrones"  # a show we know exist
            await test_single_api_call(session, invalid_url, "q", show_name)
        except aiohttp.ClientError as e:
            assert True
        
                
# run test        
await testing_suite()


In [ ]:
import asyncio
import aiohttp
from http.client import responses  
import signal

## Define a signal handler function to handle the shutdown signal ##
def shutdown_handler(signum, frame):
    print(f"Received the shutdown signal. Shutting down gracefully.")

## create the signal handler for the shutdown signal (SIGINT) ##
signal.signal(signal.SIGINT, shutdown_handler)

### below we modify the test_single_api_call function above to collect the information ###

## a function for making a blocking api request ##
async def blocking_api_call(session, url, parameter, blocking_name):
    param = {parameter: blocking_name}
    print("Initiating a blocking call.")
    await asyncio.sleep(5)
    try:
        response = await session.get(url=url, params=param)
        if response is not None:
            if response.status == 200:
                print(f"Status {response.status}: Request OK for {blocking_name}")
                answer.append(await response.json())

            else:
                # print the status code and what it means
                print(f"Error {response.status}: {responses[response.status]} ")

    except aiohttp.ClientError as e:
        print(f"Error making API request for {blocking_name}: {e}.")
        
    
## a function for making the non-blocking request ##
async def single_api_call(session, url, parameter, name, answer):
    param = {parameter: name}
    try: 
        response = await session.get(url=url, params=param)  
        if response.status == 200:
            print(f"Status {response.status}: Request OK for {name}")
            answer.append(await response.json())

        else:
            # print the status code and what it means
            print(f"Error {response.status}: {responses[response.status]} ")    
    except aiohttp.ClientError as e:
        print(f"Error making API request for {name}: {e}.")
                

## collect the non-blocking request ##   
def create_api_call_list(session, url, parameter, values, answer):
    tasks = []
    for name in values:
        tasks.append(single_api_call(session, url, parameter, name, answer))
    return(tasks)

## create a session for running the requests a asynchronously ##        
async def main(): 
    async with aiohttp.ClientSession() as session:
        tasks = create_api_call_list(session, url, "q", values, answer)
        responses = await asyncio.gather(blocking_api_call(session, url, "q", "the big bang theory"), *tasks)
    

## The site for making api requests contains jsons for TV shows. ##
url = "https://api.tvmaze.com/singlesearch/shows"

## We will make a (non-blocking) API requests for the following list of shows ##
values = ["game of thrones", "girls", "the sopranos", "the chef show", "non existent", "the bear", "the morning show"]

## This will be used for collecting the information ##
answer = []
        
# execute the code above 
await main()

print("The program is completed.")